In [96]:
import pandas as pd
import numpy as np

# --- Загрузка данных ---
fin_df = pd.read_csv("financial_data.csv")
#prol_df = pd.read_csv("prolongations.csv")

prol_df  = pd.read_csv("prolongations.csv", header=0, 
                         names=['id','last_month','account_name'])

In [97]:
fin_df.head(20)

,id,Причина дубля,Ноябрь 2022,Декабрь 2022,Январь 2023,Февраль 2023,Март 2023,Апрель 2023,Май 2023,Июнь 2023,Июль 2023,Август 2023,Сентябрь 2023,Октябрь 2023,Ноябрь 2023,Декабрь 2023,Январь 2024,Февраль 2024,Account
0,42,NaN,"36 220,00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Васильев Артем Александрович
1,657,первая часть оплаты,стоп,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Васильев Артем Александрович
2,657,вторая часть оплаты,стоп,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Васильев Артем Александрович
3,594,NaN,стоп,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Васильев Артем Александрович
4,665,NaN,"10 000,00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Васильев Артем Александрович
5,637,NaN,"38 045,00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Соколова Анастасия Викторовна
6,419,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Михайлов Андрей Сергеевич
7,578,NaN,"82 800,00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Попова Екатерина Николаевна
8,579,NaN,в ноль,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Васильев Артем Александрович
9,592,первая часть оплаты,"92 302,00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Васильев Артем Александрович


In [98]:
# --- Определяем колонки месяцев ---
fin_df.columns = [col.lower() for col in fin_df.columns]
non_month_cols = ["id", "причина дубля", "account"]
month_cols = [col for col in fin_df.columns if col not in non_month_cols]

In [99]:
# словарь для русских месяцев
months_map = {
    "январь": 1, "февраль": 2, "март": 3, "апрель": 4,
    "май": 5, "июнь": 6, "июль": 7, "август": 8,
    "сентябрь": 9, "октябрь": 10, "ноябрь": 11, "декабрь": 12
}

def parse_ru_month(s):
    """Парсинг строки вида 'Ноябрь 2022' → datetime(2022, 11, 1)"""
    name, year = s.split()
    month = months_map[name.lower()]
    return pd.to_datetime(f"{year}-{month:02d}-01")

# --- Сопоставляем месяцы в financial_data ---
month_map = {col: parse_ru_month(col) for col in month_cols}
month_cols_sorted = sorted(month_cols, key=lambda c: month_map[c])

# --- Преобразуем last_month в datetime ---
prol_df["last_month_dt"] = prol_df["last_month"].apply(parse_ru_month)

In [100]:
# --- Чистим данные от "стоп"/"end" ---

mask_stop = fin_df.apply(lambda row: any(val in ["стоп", "end"] for val in row[month_cols]), axis=1)
fin_df = fin_df[~mask_stop]


for col in month_cols:
    fin_df[col] = (
        fin_df[col]
        .apply(lambda x: 0 if x == 'в ноль' else x)
        .str.replace('\xa0', '', regex=False)
        .str.replace(",", ".", regex=False)
        .astype(float)
    )


fin_df = fin_df[fin_df[month_cols].sum(axis=1) != 0]

In [101]:
# --- Функция для расчёта коэффициентов ---

def calc_ratios(prol, fin, month_cols_sorted):
    results = []

    for idx, month in enumerate(month_cols_sorted[1:], start=1):
        prev_month = month_cols_sorted[idx-1]
        prev_dt = month_map[prev_month]
        curr_dt = month_map[month]

        # --- Коэф. 1 (пролонгации в первый месяц) ---
        ids_prev = prol.loc[prol.last_month_dt == prev_dt, "id"]
        base_sum = fin.loc[fin.id.isin(ids_prev), prev_month].sum()
        ext_sum = fin.loc[fin.id.isin(ids_prev), month].sum()
        ratio1 = ext_sum / base_sum if base_sum > 0 else np.nan

        # --- Коэф. 2 (пролонгации во второй месяц) ---
        ids_prev2 = prol.loc[prol.last_month_dt == month_map[month_cols_sorted[idx-2]], "id"] if idx >= 2 else []
        if len(ids_prev2) > 0:
            mask = fin.id.isin(ids_prev2)
            base_sum2 = fin.loc[mask & (fin[prev_month] == 0), month_cols_sorted[idx-2]].sum()
            ext_sum2 = fin.loc[mask & (fin[prev_month] == 0), month].sum()
            ratio2 = ext_sum2 / base_sum2 if base_sum2 > 0 else np.nan
        else:
            ratio2 = np.nan

        results.append({
            "Месяц": month,
            "Коэф_1": ratio1,
            "Коэф_2": ratio2
        })

    return pd.DataFrame(results)

In [102]:
# --- Расчёт по каждому аккаунт-менеджеру ---
managers = []
for am in prol_df['account_name'].unique():
    m_prol = prol_df[prol_df['account_name'] == am]
    m_fin = fin_df[fin_df['Account'] == am]
    df = calc_ratios(m_prol, m_fin, month_cols_sorted)
    df["Менеджер"] = am
    managers.append(df)

am_ratios = pd.concat(managers, ignore_index=True)

KeyError: 'Account'

In [104]:
# --- Расчёт по всему отделу ---
dept_ratios = calc_ratios(prol_df, fin_df, month_cols_sorted)
dept_ratios

,Месяц,Коэф_1,Коэф_2
0,декабрь 2022,0.679578,NaN
1,январь 2023,0.443607,NaN
2,февраль 2023,0.764581,NaN
3,март 2023,0.626968,NaN
4,апрель 2023,0.414892,NaN
5,май 2023,0.494897,NaN
6,июнь 2023,0.249892,NaN
7,июль 2023,0.521927,NaN
8,август 2023,0.487207,NaN
9,сентябрь 2023,0.321457,NaN


In [105]:
# --- Итоги за год ---
dept_year = dept_ratios[["Коэф_1", "Коэф_2"]].mean()
am_year = am_ratios.groupby("Менеджер")[["Коэф_1", "Коэф_2"]].mean()

NameError: name 'am_ratios' is not defined